In [ ]:
!apt-get update
!apt-get install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 388 kB in 1s (291 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
R

In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 25.8 MB/s eta 0:00:00


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time

chrome_options = Options()
chrome_options.add_argument('--headless')  # Ejecuta sin interfaz gráfica
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Remove 'chromedriver' to avoid specifying options twice
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
driver.get("https://contrataciondelestado.es/wps/portal/licitaciones")
time.sleep(3)  # Esperar a que cargue la página


In [ ]:
try:
    boton_licitaciones = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:linkFormularioBusqueda"))
    )
    # 3. Hacer clic en el botón
    boton_licitaciones.click()
    time.sleep(3)  # Pausa para que cargue el formulario
    print("Se hizo clic en el botón y se cargó el formulario.")
except:
    print("No se encontró el botón o no se pudo hacer clic.")

# 4. (Opcional) Verificar la URL o el contenido después de hacer clic
print("URL actual:", driver.current_url)

Se hizo clic en el botón y se cargó el formulario.
URL actual: https://contrataciondelestado.es/wps/portal/!ut/p/b1/04_Sj9Q1NDE0MjY2NDPXj9CPykssy0xPLMnMz0vMAfGjzOJNXP2dnd08jAwsgo1dDIx8XEJDzTyAXHdToIJIfAqMiNNvauxsEuYVFmAW7OluYODp4ebiE2poCtRuRpx-AxzA0YCQ_nD9KPxKjKAK8DkRrACPG_w88nNT9XOjctzcLD2zTBwVFQG-bRT6/dl4/d5/L2dBISEvZ0FBIS9nQSEh/pw/Z7_AVEQAI930OBRD02JPMTPG21004/act/id=0/p=javax.servlet.include.path_info=QCPjspQCPbusquedaQCPMainBusqueda.jsp/603358685978/-/


In [ ]:
# 2. Localizar el <select> de Financiación UE y seleccionar "Sí"
try:
    # Espera explícita a que el <select> aparezca
    wait = WebDriverWait(driver, 10)
    financiacion_select_element = wait.until(
        EC.presence_of_element_located(
            (By.ID, "viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:Financiacion")
        )
    )

    # Crear un objeto Select para manejar el <select>
    financiacion_select = Select(financiacion_select_element)
    # Seleccionar la opción "Sí" (que corresponde a value="EU" o texto visible "Sí")
    financiacion_select.select_by_visible_text("Sí")

    # 3. Localizar el botón "Buscar" y hacer clic
    buscar_button = driver.find_element(By.ID, "viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:button1")
    buscar_button.click()

    time.sleep(3)  # Esperar a que cargue el resultado
    print("Búsqueda realizada con Financiación UE = Sí.")

except Exception as e:
    print("Ocurrió un error:", e)

# 4. (Opcional) Procesar los resultados
html_resultado = driver.page_source
print("Longitud del HTML resultado:", len(html_resultado))

#driver.quit()

Búsqueda realizada con Financiación UE = Sí.
Longitud del HTML resultado: 251040


In [ ]:
import time
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

# Lista para guardar todas las licitaciones
licitaciones = []

while True:
    # Esperamos a que se carguen las filas de la tabla (ajusta el timeout según necesites)
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//tbody/tr")))
    time.sleep(2)  # Pequeña espera adicional para que se asiente el contenido

    # Obtenemos el HTML de la página y lo parseamos
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Se asume que la tabla está dentro de un <tbody>
    table_body = soup.find("tbody")
    if not table_body:
        print("No se encontró la tabla de licitaciones.")
        break

    # Extraemos todas las filas
    filas = table_body.find_all("tr")

    for fila in filas:
        # Extraer información de cada celda
        # 1. Número de expediente y descripción (dentro de la celda con class "tdExpediente")
        td_expediente = fila.find("td", class_="tdExpediente")
        if td_expediente:
            # El número lo contiene el primer <span>
            span_num = td_expediente.find("span")
            numero = span_num.get_text(strip=True) if span_num else ""
            # La descripción suele estar en el segundo <div>
            divs = td_expediente.find_all("div")
            descripcion = divs[1].get_text(strip=True) if len(divs) > 1 else ""
        else:
            numero = ""
            descripcion = ""

        # 2. Tipo de contrato (celda con class "tdTipoContrato")
        td_tipo = fila.find("td", class_="tdTipoContrato")
        if td_tipo:
            div_tipo = td_tipo.find_all("div")
            tipo1 = div_tipo[0].get_text(strip=True) if len(div_tipo) > 0 else ""
            tipo2 = div_tipo[1].get_text(strip=True) if len(div_tipo) > 1 else ""
        else:
            tipo1 = ""
            tipo2 = ""

        # 3. Estado (celda con class "tdEstado")
        td_estado = fila.find("td", class_="tdEstado")
        estado = td_estado.get_text(strip=True) if td_estado else ""

        # 4. Importe (celda con class "tdImporte")
        td_importe = fila.find("td", class_="tdImporte")
        importe = td_importe.get_text(strip=True) if td_importe else ""

        # 5. Fecha límite (celda con class "tdFechaLimite")
        td_fecha = fila.find("td", class_="tdFechaLimite")
        fecha_limite = td_fecha.get_text(strip=True) if td_fecha else ""

        # 6. Órgano de contratación (celda con class "tdOrganoContratacion")
        td_organo = fila.find("td", class_="tdOrganoContratacion")
        organo = td_organo.get_text(strip=True) if td_organo else ""

        # Almacenamos la licitación en un diccionario
        licitacion = {
            "Numero": numero,
            "Descripcion": descripcion,
            "TipoContrato1": tipo1,
            "TipoContrato2": tipo2,
            "Estado": estado,
            "Importe": importe,
            "FechaLimite": fecha_limite,
            "Organo": organo
        }
        licitaciones.append(licitacion)

    print(f"Extraídas {len(filas)} licitaciones de la página actual.")

    # Intentar hacer clic en el botón "Siguiente" para pasar a la siguiente página
    try:
        siguiente_btn = driver.find_element(By.ID, "viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:footerSiguiente")
        # Si el botón no está habilitado o no se encuentra, se asume que es la última página.
        if siguiente_btn.is_enabled():
            siguiente_btn.click()
            # Esperar a que se cargue la nueva página
            time.sleep(3)
        else:
            print("Botón 'Siguiente' deshabilitado. Última página alcanzada.")
            break
    except Exception as e:
        print("No se encontró el botón 'Siguiente' o ha ocurrido un error:", e)
        break

# Cerrar el driver
driver.quit()

# Guardar los datos en un archivo CSV
nombre_archivo = "licitaciones.csv"
with open(nombre_archivo, "w", newline="", encoding="utf-8") as f:
    campo_nombres = ["Numero", "Descripcion", "TipoContrato1", "TipoContrato2", "Estado", "Importe", "FechaLimite", "Organo"]
    writer = csv.DictWriter(f, fieldnames=campo_nombres)
    writer.writeheader()
    writer.writerows(licitaciones)

print(f"Se han guardado {len(licitaciones)} licitaciones en el archivo {nombre_archivo}.")


Extraídas 20 licitaciones de la página actual.
Extraídas 20 licitaciones de la página actual.
Extraídas 20 licitaciones de la página actual.
Extraídas 20 licitaciones de la página actual.
Extraídas 20 licitaciones de la página actual.
Extraídas 20 licitaciones de la página actual.
Extraídas 20 licitaciones de la página actual.
Extraídas 20 licitaciones de la página actual.
Extraídas 20 licitaciones de la página actual.
Extraídas 20 licitaciones de la página actual.
Extraídas 20 licitaciones de la página actual.
Extraídas 20 licitaciones de la página actual.
Extraídas 20 licitaciones de la página actual.
Extraídas 20 licitaciones de la página actual.
Extraídas 20 licitaciones de la página actual.
Extraídas 20 licitaciones de la página actual.
Extraídas 20 licitaciones de la página actual.
Extraídas 20 licitaciones de la página actual.
Extraídas 20 licitaciones de la página actual.
Extraídas 20 licitaciones de la página actual.
Extraídas 20 licitaciones de la página actual.
Extraídas 20 

# DEPURACION


In [ ]:
driver.save_screenshot("pantalla.png")


True

In [ ]:
driver.quit()

In [ ]:
driver.get("https://contrataciondelestado.es/wps/portal/licitaciones")
time.sleep(5)  # Espera un poco

# Muestra el HTML que Selenium realmente está recibiendo
page_source = driver.page_source
print(page_source)


In [ ]:
import time
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

# Lista para almacenar los datos de licitaciones
licitaciones = []

# Número de páginas a recorrer (en este ejemplo, 2)
num_paginas = 2

for pagina in range(num_paginas):
    print(f"Procesando página {pagina+1}...")
    # Esperar a que se carguen las filas de la tabla
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//tbody/tr")))
    time.sleep(2)  # Pequeña espera adicional

    # Parsear el HTML de la página
    soup = BeautifulSoup(driver.page_source, "html.parser")
    table_body = soup.find("tbody")
    if not table_body:
        print("No se encontró la tabla de licitaciones en la página", pagina+1)
        break

    filas = table_body.find_all("tr")

    for fila in filas:
        # Extraer información de la celda de Expediente
        td_expediente = fila.find("td", class_="tdExpediente")
        if td_expediente:
            # Se asume que el número de expediente está en el primer <span>
            span_num = td_expediente.find("span")
            numero = span_num.get_text(strip=True) if span_num else ""
            # La descripción suele estar en el segundo <div>
            divs = td_expediente.find_all("div")
            descripcion = divs[1].get_text(strip=True) if len(divs) > 1 else ""
        else:
            numero = ""
            descripcion = ""

        # Extraer Tipo de Contrato
        td_tipo = fila.find("td", class_="tdTipoContrato")
        if td_tipo:
            div_tipo = td_tipo.find_all("div")
            tipo1 = div_tipo[0].get_text(strip=True) if len(div_tipo) > 0 else ""
            tipo2 = div_tipo[1].get_text(strip=True) if len(div_tipo) > 1 else ""
        else:
            tipo1 = ""
            tipo2 = ""

        # Extraer Estado
        td_estado = fila.find("td", class_="tdEstado")
        estado = td_estado.get_text(strip=True) if td_estado else ""

        # Extraer Importe
        td_importe = fila.find("td", class_="tdImporte")
        importe = td_importe.get_text(strip=True) if td_importe else ""

        # Extraer Fecha Límite
        td_fecha = fila.find("td", class_="tdFechaLimite")
        fecha_limite = td_fecha.get_text(strip=True) if td_fecha else ""

        # Extraer Órgano de Contratación
        td_organo = fila.find("td", class_="tdOrganoContratacion")
        organo = td_organo.get_text(strip=True) if td_organo else ""

        # Almacenar la licitación en un diccionario
        licitacion = {
            "Numero": numero,
            "Descripcion": descripcion,
            "TipoContrato1": tipo1,
            "TipoContrato2": tipo2,
            "Estado": estado,
            "Importe": importe,
            "FechaLimite": fecha_limite,
            "Organo": organo
        }
        licitaciones.append(licitacion)

    print(f"Extraídas {len(filas)} licitaciones de la página {pagina+1}.")

    # Si no es la última página a procesar, hacer clic en "Siguiente"
    if pagina < num_paginas - 1:
        try:
            siguiente_btn = driver.find_element(By.ID, "viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:footerSiguiente")
            if siguiente_btn.is_enabled():
                siguiente_btn.click()
                time.sleep(3)  # Esperar a que cargue la siguiente página
            else:
                print("Botón 'Siguiente' deshabilitado en la página", pagina+1)
                break
        except Exception as e:
            print("Error al pulsar el botón 'Siguiente' en la página", pagina+1, ":", e)
            break

# Cerrar el driver
driver.quit()

# Guardar los datos extraídos en un archivo CSV
nombre_archivo = "licitaciones_2paginas.csv"
with open(nombre_archivo, "w", newline="", encoding="utf-8") as f:
    campos = ["Numero", "Descripcion", "TipoContrato1", "TipoContrato2", "Estado", "Importe", "FechaLimite", "Organo"]
    writer = csv.DictWriter(f, fieldnames=campos)
    writer.writeheader()
    writer.writerows(licitaciones)

print(f"Se han guardado {len(licitaciones)} licitaciones en el archivo '{nombre_archivo}'.")


Procesando página 1...
Extraídas 20 licitaciones de la página 1.
Procesando página 2...
Extraídas 20 licitaciones de la página 2.
Se han guardado 40 licitaciones en el archivo 'licitaciones_2paginas.csv'.
